In [1]:
import os

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

import json

In [2]:
directory_in = './file/2.part_number'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'1.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：450



,No,Part Number,Url
0,1,APK100,https://www.standardbrand.com/en/ecatalog?part...
1,2,APK101,https://www.standardbrand.com/en/ecatalog?part...
2,3,APK102,https://www.standardbrand.com/en/ecatalog?part...
3,4,APK103,https://www.standardbrand.com/en/ecatalog?part...
4,5,APS101,https://www.standardbrand.com/en/ecatalog?part...
...,...,...,...
445,446,APS717,https://www.standardbrand.com/en/ecatalog?part...
446,447,G92003,https://www.standardbrand.com/en/ecatalog?part...
447,448,G92004,https://www.standardbrand.com/en/ecatalog?part...
448,449,G92005,https://www.standardbrand.com/en/ecatalog?part...


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                             1
 Part Number                                               APK100
 Url            https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object)

In [5]:
import time

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part={crawler_series['Part Number']}&func=PART&vid=''', crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            dict_dict = resp.json()

            if dict_dict:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_dict

1


{'pd': {'rowId': 1,
  'site': 'STD',
  'webBase': 'APK100',
  'brand': 'STD',
  'basePart': 'APK100',
  'partDesc_en': 'Accelerator Pedal Sensor',
  'partLongDesc_en': 'Accelerator Pedal Sensor',
  'pop': 'D',
  'partComment': None,
  'qtyEach': 1,
  'webColumn_Id': None,
  'categoryName_en': None,
  'vehicleId': None,
  'mfg': None,
  'mfG_Part': None,
  'maxRows': 1,
  'brandLink': None,
  'imageUrl': None},
 'pp': {'buyersGuides': [{'basePart': 'APK100',
    'buyersGuideId': 49562,
    'buyersGuideDesc': 'Cadillac Escalade (03)'},
   {'basePart': 'APK100',
    'buyersGuideId': 49800,
    'buyersGuideDesc': 'Chevrolet Avalanche 1500 (03)'},
   {'basePart': 'APK100',
    'buyersGuideId': 5877,
    'buyersGuideDesc': 'Chevrolet Avalanche 2500 (03)'},
   {'basePart': 'APK100',
    'buyersGuideId': 11188,
    'buyersGuideDesc': 'Chevrolet Silverado 1500 (03)'},
   {'basePart': 'APK100',
    'buyersGuideId': 51639,
    'buyersGuideDesc': 'Chevrolet Silverado 1500 HD (03)'},
   {'basePart'

In [6]:
dict_info = {key.strip().title(): value.strip() if type(value) == str else value for key, value in dict_dict['pd'].items() if key not in ['rowId', 'site', 'webBase', 'basePart', 'maxRows'] and value != None}

dict_info

{'Brand': 'STD',
 'Partdesc_En': 'Accelerator Pedal Sensor',
 'Partlongdesc_En': 'Accelerator Pedal Sensor',
 'Pop': 'D',
 'Qtyeach': 1}

In [7]:
vehicle = '\n'.join([dict_['buyersGuideDesc'].strip() for dict_ in dict_dict['pp']['buyersGuides']])

print(vehicle)

Cadillac Escalade (03)
Chevrolet Avalanche 1500 (03)
Chevrolet Avalanche 2500 (03)
Chevrolet Silverado 1500 (03)
Chevrolet Silverado 1500 HD (03)
Chevrolet Silverado 2500 (03)
Chevrolet Silverado 2500 HD (05-03)
Chevrolet Silverado 3500 (05-03)
Chevrolet Suburban 1500 (03)
Chevrolet Suburban 2500 (03)
Chevrolet Tahoe (03)
GMC Sierra 1500 (03-01)
GMC Sierra 1500 HD (03-01)
GMC Sierra 2500 (03-00)
GMC Sierra 2500 HD (05-01)
GMC Sierra 3500 (05-01)
GMC Yukon (03-00)
GMC Yukon XL 1500 (03-00)
GMC Yukon XL 2500 (03-00)
Hummer H2 (03)


In [8]:
dict_key, list_dict = {}, dict_dict['pp']['partSpecs']
for dict_ in list_dict:
    key = dict_['attributeName_en'].strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_specification, dict_count = {}, {}
for i, dict_ in enumerate(list_dict):
    key = dict_['attributeName_en'].strip()
    if dict_key[key] == 1:
        dict_specification[str(i)] = {key: dict_['siteValue_en'].strip()}
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_specification[str(i)] = {f'{key}-{dict_count[key]}': dict_['siteValue_en'].strip()}

dict_specification

{'0': {'Warranty': '3 years/36,000 mile'},
 '1': {'New or Reman': 'New'},
 '2': {'Linkage Attached': 'No'},
 '3': {'Pedal Assembly Included': 'Yes'},
 '4': {'Connector Quantity': '1'},
 '5': {'Connector Shape': 'Rectangular'},
 '6': {'Connector Gender': 'Female'},
 '7': {'Terminal Quantity': '9'},
 '8': {'Terminal Gender': 'Male (1)'},
 '9': {'Terminal Type': 'Pin'},
 '10': {'Attachment Method': 'Bolt On'},
 '11': {'Color/Finish': 'Black'},
 '12': {'Hardware included': 'No'},
 '13': {'Material': 'Polyamide 6'},
 '14': {'Mounting Bracket(s) Included': 'No'}}

In [9]:
dict_type, list_dict = {}, dict_dict['pp']['categories']
for i, dict_ in enumerate(list_dict):
    dict_type[str(i)] = dict_['categoryName_en'].strip()

dict_type

{'0': 'Sensor - Accelerator Pedal'}

In [10]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Part Number': crawler_series['Part Number'],
                          'Vehicle': vehicle,
                          'Picture': '',
                          'Url': crawler_series['Url'],
                          'Json_Type': json.dumps(dict_type),
                          'Json_Info': json.dumps(dict_info),
                          'Json_Specification': json.dumps(dict_specification)})

list_dict_correct

[{'No': 1,
  'Part Number': 'APK100',
  'Vehicle': 'Cadillac Escalade (03)\nChevrolet Avalanche 1500 (03)\nChevrolet Avalanche 2500 (03)\nChevrolet Silverado 1500 (03)\nChevrolet Silverado 1500 HD (03)\nChevrolet Silverado 2500 (03)\nChevrolet Silverado 2500 HD (05-03)\nChevrolet Silverado 3500 (05-03)\nChevrolet Suburban 1500 (03)\nChevrolet Suburban 2500 (03)\nChevrolet Tahoe (03)\nGMC Sierra 1500 (03-01)\nGMC Sierra 1500 HD (03-01)\nGMC Sierra 2500 (03-00)\nGMC Sierra 2500 HD (05-01)\nGMC Sierra 3500 (05-01)\nGMC Yukon (03-00)\nGMC Yukon XL 1500 (03-00)\nGMC Yukon XL 2500 (03-00)\nHummer H2 (03)',
  'Picture': '',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=APK100&type=p&search=s',
  'Json_Type': '{"0": "Sensor - Accelerator Pedal"}',
  'Json_Info': '{"Brand": "STD", "Partdesc_En": "Accelerator Pedal Sensor", "Partlongdesc_En": "Accelerator Pedal Sensor", "Pop": "D", "Qtyeach": 1}',
  'Json_Specification': '{"0": {"Warranty": "3 years/36,000 mile"}, "1": {"New or Reman"

In [11]:
crawler_status = True

crawler_status

True

In [12]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Part Number                                               APK100
 Url            https://www.standardbrand.com/en/ecatalog?part...
 Request_Url    https://ecatalog.smpcorp.com/V2/STD/api/part/p...
 Check_Url      https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object]

In [13]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Part Number,Vehicle,Picture,Url,Json_Type,Json_Info,Json_Specification
0,1,APK100,Cadillac Escalade (03)\nChevrolet Avalanche 15...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [14]:
print('Output Finished !')
print()

Output Finished !



In [15]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Part Number,Url,Request_Url,Check_Url
0,1,APK100,https://www.standardbrand.com/en/ecatalog?part...,https://ecatalog.smpcorp.com/V2/STD/api/part/p...,https://www.standardbrand.com/en/ecatalog?part...


In [16]:
print('Done ~')

Done ~
